SPLIT DATA (WINDOWING)

In [3]:
from src.execution import WindowSplittingExecution

In [4]:
DATA: str = "assets/synthetic-data-offset.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.2
BATCH_SPLIT: bool = False 
BATCHSIZE: int = 326 # real-data: 1800, synthetic-data: 326
SEED: int = 42 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "." # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)

Anzahl an Bleche in der Gesamtmatrix 182
(5752, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (46009, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (5752, 10, 11) 
Shape für die Labels der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (46009, 1, 3)
Shape für die Labels der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (5752, 1, 3)
Data saved in build/window_split/synthetic-data-offset/1742239499


TUNE MODEL

In [5]:
from src.execution import CNNTuningExecution

In [6]:
SPLITTED_DATA_FOLDER = "build/window_split/synthetic-data-offset/1742239499"

CNNTuningExecution.execute(SPLITTED_DATA_FOLDER)

Trial 30 Complete [00h 01m 10s]
val_loss: 2.7855777740478516

Best val_loss So Far: 0.7809644937515259
Total elapsed time: 00h 49m 35s
All available hyperparameters:
{'num_layers_conv': 0, 'learning_rate': 0.00029335291375872573, 'num_layers_fully': 4, 'units_dense0': 192, 'activation_dense0': 'tanh', 'l2_dense0': 0.001, 'units_conv0': 2, 'activation_conv0': 'relu', 'kernel_0': 5, 'l2_conv0': 0.004, 'units_conv1': 2, 'activation_conv1': 'relu', 'kernel_1': 3, 'l2_conv1': 0.003, 'units_conv2': 7, 'activation_conv2': 'tanh', 'kernel_2': 5, 'l2_conv2': 0.004, 'units_conv3': 14, 'activation_conv3': 'tanh', 'kernel_3': 2, 'l2_conv3': 0.0, 'units_conv4': 13, 'activation_conv4': 'relu', 'kernel_4': 3, 'l2_conv4': 0.001, 'units_dense1': 512, 'activation_dense1': 'relu', 'l2_dense1': 0.004, 'units_dense2': 480, 'activation_dense2': 'relu', 'l2_dense2': 0.007, 'units_dense3': 320, 'activation_dense3': 'tanh', 'l2_dense3': 0.01, 'units_dense4': 448, 'activation_dense4': 'tanh', 'l2_dense4': 0.001

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 110)       │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 192)       │     21,312 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 512)       │     98,816 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 480)       │    246,240 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 320)       │    153,920 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 1)         │        321 │ dense_16[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 1)         │        321 │ dense_16[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 1)         │        321 │ dense_16[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 521,251 (1.99 MB)

 Trainable params: 521,251 (1.99 MB)

 Non-trainable params: 0 (0.00 B)

TRAIN MODEL

In [7]:
from src.execution import CNNTrainingExecution

In [8]:
TUNED_MODEL_FILE = "build/tune/synthetic-data-offset/1742239520/cnn-synthetic-data-offset.h5"
DATA_FILE = "build/window_split/synthetic-data-offset/1742239499"
SAVE_FILENMAME = "cnn-synthetic-offset.h5" # With file extension
LEARNING_RATE = 0.00029335291375872573

CNNTrainingExecution.execute(TUNED_MODEL_FILE, DATA_FILE, SAVE_FILENMAME, LEARNING_RATE)

Epoch 1/30
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - Verstellweg_Phi_loss: 0.4271 - Verstellweg_Phi_mae: 0.4271 - Verstellweg_X_loss: 0.0853 - Verstellweg_X_mae: 0.0853 - Verstellweg_Y_loss: 1.5924 - Verstellweg_Y_mae: 1.5924 - loss: 7.4681 - val_Verstellweg_Phi_loss: 0.2248 - val_Verstellweg_Phi_mae: 0.2249 - val_Verstellweg_X_loss: 0.0340 - val_Verstellweg_X_mae: 0.0340 - val_Verstellweg_Y_loss: 0.9680 - val_Verstellweg_Y_mae: 0.9684 - val_loss: 3.1941
Epoch 2/30
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - Verstellweg_Phi_loss: 0.1886 - Verstellweg_Phi_mae: 0.1886 - Verstellweg_X_loss: 0.0315 - Verstellweg_X_mae: 0.0315 - Verstellweg_Y_loss: 0.9462 - Verstellweg_Y_mae: 0.9462 - loss: 2.8039 - val_Verstellweg_Phi_loss: 0.2211 - val_Verstellweg_Phi_mae: 0.2210 - val_Verstellweg_X_loss: 0.0246 - val_Verstellweg_X_mae: 0.0246 - val_Verstellweg_Y_loss: 0.8795 - val_Verstellweg_Y_mae: 0.8800 - val_loss: 2.1285
Epoch 3/30
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - Verstellweg_Ph

VALIDATE MODEL

In [9]:
from src.execution import CNNValidationExecution 

In [10]:
MODEL_FOLDER = "build/train/cnn-synthetic-data-offset/1742244525"
DATA_FOLDER = "build/window_split/real-data/1741957181"

CNNValidationExecution.execute(MODEL_FOLDER, DATA_FOLDER)

3575/3575 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - Verstellweg_Phi_loss: 2.4505 - Verstellweg_Phi_mean_absolute_error: 2.4505 - Verstellweg_X_loss: 0.1676 - Verstellweg_X_mean_absolute_error: 0.1676 - Verstellweg_Y_loss: 3.9079 - Verstellweg_Y_mean_absolute_error: 3.9079 - loss: 6.7842


In [ ]:
import keras

# Load your pre-trained model
model = keras.models.load_model("build/train/cnn-synthetic/1741910385/cnn.h5")  # Replace with your file path

# Print model summary to understand the layer structure
model.summary()

# Freeze all convolutional and pooling layers
for layer in model.layers[:3]:  # Freezing first 3 layers (Conv1D + MaxPooling + Flatten)
    layer.trainable = False

# Keep the Dense layers trainable
for layer in model.layers[3:]:  # Unfreezing dense layers
    layer.trainable = True

# Print trainable status of layers
for layer in model.layers:
    print(layer.name, "Trainable" if layer.trainable else "Frozen")


model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
              loss="mse",  # Assuming regression output
              metrics=["mae"])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 10, 6)     │        138 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 5, 6)      │          0 │ conv1d_4[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 30)        │          0 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 352)       │     10,912 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 320)       │    112,960 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 224)       │     71,904 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │     57,600 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 416)       │    106,912 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 448)       │    186,816 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 1)         │        449 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 1)         │        449 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 1)         │        449 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 548,591 (2.09 MB)

 Trainable params: 548,589 (2.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

input_layer_1 Frozen
conv1d_4 Frozen
max_pooling1d_4 Frozen
flatten_1 Trainable
dense_6 Trainable
dense_7 Trainable
dense_8 Trainable
dense_9 Trainable
dense_10 Trainable
dense_11 Trainable
Verstellweg_X Trainable
Verstellweg_Y Trainable
Verstellweg_Phi Trainable
